In [ ]:
# 1. 네이버 드라이브 내 파일명 변환

# 문서 번호_파일명

import os
import pandas as pd

def rename_local_files_with_doc_id(csv_path, local_root_folder):
    """
    CSV에 정의된 파일명 및 확장자와 로컬 파일을 비교해,
    일치하는 경우 해당 파일에 문서번호를 붙여서 이름을 변경함.
    
    Parameters:
    - csv_path (str): CSV 파일 경로
    - local_root_folder (str): 로컬 파일들이 존재하는 루트 폴더 경로
    """
    df = pd.read_csv(csv_path)

    for idx, row in df.iterrows():
        doc_id = str(row['문서번호'])
        file_name = row['파일명']
        extension = row['확장자']
        
        expected_filename = f"{file_name}.{extension}"
        
        for root, dirs, files in os.walk(local_root_folder):
            if expected_filename in files:
                original_path = os.path.join(root, expected_filename)
                new_filename = f"{doc_id}_{file_name}.{extension}"
                new_path = os.path.join(root, new_filename)
                
                # 파일 이름 변경
                os.rename(original_path, new_path)
                print(f"Renamed: {original_path} -> {new_path}")

In [ ]:
rename_local_files_with_doc_id("파일정보.csv", "C:/Users/username/Documents/파일들")

In [ ]:
# 2. 로컬 이사 완료 후 변수명 변경

# 문서 번호_제품명_용량_상위 폴더명_최종 수정일


In [ ]:
# 3. 문서 번호 매치된 CSV 파일 생성

# ** 매치되지 않은 파일은 그대로 > 이사 제외 사유 기입